# Mongo Agent

In [2]:
from smolagents.agents import ToolCallingAgent
from smolagents import tool, LiteLLMModel, CodeAgent
from pymongo import MongoClient
from datetime import datetime
from typing import List, Dict, Optional

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL_ID = "gpt-4o-mini"
MONGODB_URI = os.getenv("MONGO_URI")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
debug = os.getenv('DEBUG', 'False').lower() == 'true'

In [5]:
mongoclient = MongoClient(MONGODB_URI, appname="mongo-agent")
db = mongoclient.warehouse